In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('senti.csv')
data

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,agent_tone,customer_tone,average_sentiment,...,day_of_week,cleaned_transcript,handle_time,primary_call_reason_cleaned,call_type_frequency,agent_dialogues,customer_dialogues,agent_sentiment_score,customer_sentiment_score,busy_period
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,16698,21830,-0.04,...,2,Agent: Thank you for calling United Airlines c...,31.0,18,1607,Thank you for calling United Airlines customer...,"Hi, yeah I'm calling because my flight from Ch...",0.316018,0.019780,0
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,54164,34574,0.02,...,3,"Agent: Thank you for calling United Airlines, ...",12.0,2,2637,"Thank you for calling United Airlines, my name...","Hi Sam, yeah I'm calling because I need to cha...",0.145596,0.268554,0
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,33823,23599,-0.13,...,2,Agent: Thank you for calling United Airlines c...,19.0,8,13311,Thank you for calling United Airlines customer...,"Hi Steve, uh, I'm calling because I need to ma...",0.244168,0.418736,0
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,24604,5178,-0.20,...,3,Agent: Thank you for calling United Airlines c...,7.0,17,2738,Thank you for calling United Airlines customer...,"Hi, yeah I'm calling because I'm having some i...",0.044670,0.047344,0
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,53222,32574,-0.05,...,3,Agent: Thank you for calling United Airlines c...,9.0,14,6365,Thank you for calling United Airlines customer...,"Hi Sarah, thanks for taking my call. I'm calli...",0.332050,0.389600,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71805,1563273072,8022667294,413148,2024-08-31 23:48:00,2024-08-31 23:57:00,2024-09-01 00:25:00,\n\nAgent: Thank you for calling United Airlin...,55166,22422,0.05,...,5,Agent: Thank you for calling United Airlines c...,28.0,11,4330,Thank you for calling United Airlines customer...,"Hi Dave, yeah I'm calling in about a flight I ...",0.273740,0.399344,0
71806,8865997781,4474867021,980156,2024-08-31 23:55:00,2024-08-31 23:58:00,2024-09-01 00:06:00,\n\nAgent: Thank you for calling United Airlin...,5848,18498,0.03,...,5,Agent: Thank you for calling United Airlines c...,8.0,17,2738,Thank you for calling United Airlines customer...,"Hi, yeah I'm calling because I'm really not ha...",0.187057,0.089015,0
71807,8019240181,9762042472,616988,2024-08-31 23:52:00,2024-08-31 23:58:00,2024-09-01 00:04:00,\n\nAgent: Thank you for calling United Airlin...,8932,33231,0.05,...,5,Agent: Thank you for calling United Airlines c...,6.0,17,2738,Thank you for calling United Airlines customer...,"Hi Steven, umm I was hoping you could help me ...",0.276867,0.242124,0
71808,8332067080,8095620266,161354,2024-08-31 23:53:00,2024-08-31 23:59:00,2024-09-01 00:03:00,\n\nAgent: Thank you for calling United Airlin...,57113,21595,-0.07,...,5,Agent: Thank you for calling United Airlines c...,4.0,0,5157,Thank you for calling United Airlines customer...,"Hi Kelly, umm.. I'm calling about a flight I h...",0.242974,0.205392,0


In [4]:
data = data.drop('cleaned_transcript', axis = 1)

In [5]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords from nltk
nltk.download('stopwords')
nltk.download('punkt')


# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# Function to clean the call transcript column
def clean_call_transcript(transcript):
    # Remove anything after double newlines "\n\n" if there's no 'Agent:' or 'Customer:' after
    transcript = re.sub(r'\n\n(?!Agent:|Customer:).*$', '', transcript, flags=re.DOTALL)
    
    # Remove redundant line breaks and extra white spaces
    transcript = re.sub(r'\n+', '\n', transcript)
    transcript = re.sub(r'\s+', ' ', transcript).strip()
    
    # Remove any trailing phrases that don't start with 'Agent:' or 'Customer:'
    lines = transcript.split('\n')
    cleaned_lines = [line for line in lines if line.startswith('Agent:') or line.startswith('Customer:')]
    cleaned_transcript = ' '.join(cleaned_lines)
    
    # Tokenize the cleaned transcript and remove stopwords
    words = word_tokenize(cleaned_transcript)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Rejoin the words to form the cleaned transcript
    cleaned_transcript = ' '.join(filtered_words)
    
    return cleaned_transcript

# Apply the function to clean the call transcript column
data['cleaned_transcript'] = data['call_transcript'].apply(clean_call_transcript)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Summary of Buckets:
- Booking and Travel Modifications: Related to managing bookings and making itinerary changes.
- Post-Flight and Customer Programs: Post-travel services, loyalty programs, and updates.
- Operational Issues and Support: Addressing real-time disruptions and specialized support needs.
- Other Inquiries and General Assistance: Miscellaneous topics and general customer inquiries.

### Advantages of These Buckets:
- Simplifies Training: Dividing the 20 categories into 4 buckets reduces the dimensionality of the output space, making it easier for a model to train with improved performance.
- Increases Generalization: Categories within each bucket share common patterns, vocabulary, and emotional tones, allowing the model to generalize better across related types of calls.
- Better Feature Representation: When processing call transcripts, the vocabulary and context of the dialogue within each bucket are likely to have similar features. This makes it easier for a natural language model to differentiate between buckets based on underlying intents.

In [4]:
# Define a dictionary to group primary call reasons into 4 buckets
call_reason_buckets = {
    "Booking and Travel Modifications": [
        "voluntary cancel", "booking", "upgrade", "voluntary change", "seating", "schedule change"
    ],
    "Post-Flight and Customer Programs": [
        "mileage plus", "post-flight", "products and services", "digital support", "traveler updates"
    ],
    "Operational Issues and Support": [
        "irrops", "check-in", "baggage", "disability", "unaccompanied minor"
    ],
    "Other Inquiries and General Assistance": [
        "checkout", "communications", "other topics", "unknown"
    ]
}

# Assign bucket labels to the primary call reasons
def assign_bucket(reason):
    for bucket, reasons in call_reason_buckets.items():
        if reason in reasons:
            return bucket
    return "Unknown"

data['bucket'] = data['primary_call_reason'].apply(assign_bucket)

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [6]:
# Prepare data for training
X = data['call_transcript'].dropna()
y = data['bucket'].dropna()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(y.unique()))

# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Tokenize data
def tokenize_data(data, tokenizer):
    return tokenizer(list(data), truncation=True, padding=True, max_length=128, return_tensors='pt')

train_encodings = tokenize_data(X_train, tokenizer)
test_encodings = tokenize_data(X_test, tokenizer)

# Convert labels to tensors
train_labels = torch.tensor(y_train.factorize()[0])
test_labels = torch.tensor(y_test.factorize()[0])

# Create Dataset objects
class CallDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = CallDataset(train_encodings, train_labels)
test_dataset = CallDataset(test_encodings, test_labels)

# Define training arguments and trainer

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Define training arguments and trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,  # Adjust batch size for GPU memory
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps',
    save_steps=1000,
    save_total_limit=2,
    fp16=True,  # Enable mixed precision training for faster GPU performance
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the moel
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn

Step,Training Loss,Validation Loss
10,1.588300,1.617185
20,1.603700,1.590611
30,1.539700,1.572990
40,1.566500,1.583556
50,1.561100,1.551442
60,1.526100,1.560701
70,1.570600,1.537967
80,1.543400,1.532889
90,1.543600,1.540148
100,1.525400,1.541434


TrainOutput(global_step=360, training_loss=1.5383708556493123, metrics={'train_runtime': 2487.8984, 'train_samples_per_second': 4.618, 'train_steps_per_second': 0.145, 'total_flos': 757780250296320.0, 'train_loss': 1.5383708556493123, 'epoch': 0.20044543429844097})

In [10]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the test dataset
predictions = trainer.predict(test_dataset)

# Get the predicted labels (converting logits to label indices)
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1).numpy()
labels = predictions.label_ids

# Calculate accuracy
accuracy = accuracy_score(labels, preds)
print(f"Test Accuracy: {accuracy:.2f}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Test Accuracy: 0.34
